In [ ]:
import os
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [ ]:
dataset_directory = '.\\dataset'

path = os.listdir(dataset_directory)

In [ ]:
'''
Reading the data using pandas dataframe

'''

labels = pd.read_csv(os.path.join(dataset_directory, 'concepts_2011.txt'), sep="\t")
image_labels = pd.read_csv(os.path.join(dataset_directory, 'trainset_gt_annotations.txt'), sep=" ", header=None)

image_labels[0] = image_labels[0].map(lambda x: x.strip('.jpg'))

list_of_features = [os.path.join(os.path.join(dataset_directory,"imageclef2011_feats") , file) for file in os.listdir(os.path.join(dataset_directory,"imageclef2011_feats")) if file.endswith(".npy") ]

In [ ]:
labels

In [ ]:
image_labels

In [ ]:
list_of_features = [os.path.join(os.path.join(dataset_directory,"imageclef2011_feats") , file) for file in os.listdir(os.path.join(dataset_directory,"imageclef2011_feats")) if file.endswith(".npy") ]

In [ ]:
feats = np.array([np.load(feat) for feat in list_of_features])

In [ ]:
x_spring = feats.transpose()[9] #Spring

In [ ]:
x_summer = feats.transpose()[10]#summer

In [ ]:
x_autumn = feats.transpose()[11] #Autumn

In [ ]:
x_winter = feats.transpose()[12] #Winter

In [ ]:
y = np.repeat(np.arange(0),8000)
y.shape

In [ ]:
'''
class based split
Spring
'''
x_train, x_test, y_train, y_test = train_test_split(x_spring, y, test_size=0.15, stratify=y )
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=2000/6800, stratify=y_train)

In [ ]:
'''
class based split
Summer
'''
x_train, x_test, y_train, y_test = train_test_split(x_summer, y, test_size=0.15, stratify=y )
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=2000/6800, stratify=y_train)

In [ ]:
'''
class based split
Fall
'''
x_train, x_test, y_train, y_test = train_test_split(x_autumn, y, test_size=0.15, stratify=y )
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=2000/6800, stratify=y_train)

In [ ]:
'''
class based split
Winter
'''
x_train, x_test, y_train, y_test = train_test_split(x_winter, y, test_size=0.15, stratify=y )
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=2000/6800, stratify=y_train)